# International College of Economics and Finance 

# Financial Econometrics. Class 04

# Forecasting

- Inspired by [Forecasting: Principles and Practice](https://otexts.com/fpp2/)

## Class outline
- Let's find the best in terms of prediction ability model for SP500

In [ ]:
# Libraries
# For those who has a problem like: package ‘package_name’ is not available (for R version x.x.x)
# install.packages('package_name', dependencies=TRUE, repos='http://cran.rstudio.com/')

library(quantmod)
library(multDM)
library(forecast) #I highly recommend to visit the link above. It explains basics of forecasting and time-series modeling in R

# For those who have operational system in Russian but wants it in English
Sys.setlocale("LC_TIME", "C")
format(Sys.Date(), format = "%Y-%b-%d")

## Download Data

In [ ]:
sp <- getSymbols("^GSPC", scr = "yahoo", auto.assign = FALSE)

In [ ]:
sp.price <- sp$GSPC.Adjusted
sp.ret <- diff(log(sp.price))[-1] * 100
names(sp.ret) <- c('S&P500 log returns')

## Models and forecasts

- Let's revisit our previous class

In [ ]:
N <- length(sp.ret) # let's have a variable with the size\length of our data
N_OOS <- round(x = 0.3 * N, digits = 0) # Usually, 25%-30% of data are used for the prediction period
N_sample <- N - N_OOS

In [ ]:
y_train <- vector(mode = 'logical', length = N_sample)
y_pred_mean <- vector(mode = 'logical', length = N_OOS)
y_pred_ar <- vector(mode = 'logical', length = N_OOS)
y_true <- sp.ret[(N_sample + 1):N]

for (i in 1:N_OOS){
    y_train <- sp.ret[i:(N_sample + i - 1)] #It is a good practice to store your in-sample data
    y_pred_mean[i] <- mean(y_train) # Because then you can just call the function on it
    ar <- lm(y_train ~ lag(y_train))
    y_pred_ar[i] <- ar$coefficients[1] + ar$coefficients[2] * y_train[N_sample]
}

y_pred_mean <- as.xts(x = y_pred_mean, order.by = index(y_true))
y_pred_ar <- as.xts(x = y_pred_ar, order.by = index(y_true))

mse_mean <- mean((y_true - y_pred_mean)^2)
mse_ar <- mean((y_true - y_pred_ar)^2)

In [ ]:
par(xpd = T)
plot(x = cbind(y_true, y_pred_mean, y_pred_ar), 
     main = 'Returns and predictions', 
     y = index(y_true), 
     col = c('black', 'red', 'blue'),
     lwd = c(1, 2, 2))
legend('bottomleft', 
       legend = c('Returns', 'Mean model predictions', 'AR(1) model predictions'), 
       col = c('black', 'red', 'blue'),
       lty = c(1, 1, 1),
       inset = 0.05)

In [ ]:
mse_mean; mse_ar

- Well, we see that AR(1) model has lower MSE
- But we want to see if its forecasts are statistically better than those of mean model

## DM test

- So, I hope that you remember the procedure
    - Make two sequence of predictions $\hat{y_1} \text{ and } \hat{y_2}$
    - Calculate the loss function (in our case it is a squared loss): $L(e_1) = (y - \hat{y_1})^2; L(e_2) = (y - \hat{y_2})^2$
    - Calculate the difference between them: $d = L(e_1) - L(e_2)$
    - If forecasts are the same: $H_0: E(d) = 0$
    - If not: $H_1: E(d) \ne 0$
    - Calculate good old t-stat: $t = \frac{\frac{1}{T}\sum_1^T d}{\sqrt{\hat{\sigma_d}/T}}=DM$
    - For estimation of $\hat{\sigma_d}$ use `HAC` estimator

In [ ]:
# Let's go
L.mean_model = (y_true - y_pred_mean)^2
L.ar = (y_true - y_pred_ar)^2

d = L.mean_model - L.ar

d.mean = mean(d)

- As for calculation of HAC estimator I will use (Newey-West, 1987)
- Also, check this [chapter](https://www.econometrics-with-r.org/15-4-hac-standard-errors.html)
- (Newey-West, 1987) HAC estimator
    - So, if our $d$ is actually not serially correlated, then: $V(\bar{d}) = V(\frac{1}{T}\sum_{t=1}^{T}d_t) = \frac{1}{T^2}\sum_{t=1}^{T}V(d_t) = \frac{1}{T}V(d_t)$. In other words it is just an unbiased variance estimator: $\hat{V}(d_t) = \frac{1}{T-1}\sum_{t=1}^{T}(d_t - \bar{d})^2$
    - Of course, we cannot say that. Hence: $V(\bar{d}) = V(\frac{1}{T}\sum_{t=1}^{T}d_t) = \frac{1}{T^2}\sum_{t=1}^{T}V(d_t) + \frac{2}{T^2}\sum_{t=1}^{T-1} \sum_{k=t+1}^{T}cov(d_t, d_k) \stackrel{why?}= \frac{1}{T^2}\sum_{t=1}^{T}V(d_t) + \frac{2}{T^2}\sum_{j=1}^{T-1} (T - j) cov(d_t, d_{t+j})$
    - It is also a good practice to truncate the sum of autocovariances. In most cases it is suggested to use $m = T^{\frac{1}{3}}$
    - Hence, Newey-West estimator is: $\frac{1}{T}\hat{V(d_t)} + \frac{2}{T}\sum_{j=1}^{m} (1 - \frac{j}{m+1}) \hat{cov}(d_t, d_{t+j})$
    - But we have some aces. This one (Diebold, F.X. and Mariano, R.S. (1995)). Authors say that the truncation lag should be $(h-1)$, where $h$ - h-step-ahead forecast. In our case, $h = 1$, meaning, that in 1-step-ahead forecast we can say that Newey-West estimator is $\frac{1}{T}\hat{V}(d_t) = \frac{1}{T} \cdot \frac{1}{T-1}\sum_{t=1}^{T}(d_t - \bar{d})$
    - Meaning that we need to use unbiased variance estimator divided by the number of observations

In [ ]:
d.var <- var(d)

In [ ]:
dm <- d.mean/sqrt(d.var/length(d))

In [ ]:
dm

In [ ]:
2 * pnorm(q = -abs(dm))

- Harvey, Leybourne, and Newbold (1997) (HLN) suggest that improved small-sample properties can be obtained by:
    - making a bias correction to the DM test statistic, and
    - comparing the corrected statistic with a Student-t distribution with (T-1) degrees of freedom, rather than the standard normal.
- The corrected statistic is obtained as:  
$$\sqrt{\frac{T + 1 - 2h + h(h-1)}{T}}\cdot DM \sim t_{T-1}$$

In [ ]:
T <- length(d)
h <- 1
k <- ((T + 1 - 2 * h + (h / T) * (h - 1)) / T) ^ (1 / 2)

In [ ]:
hln <- dm*k

In [ ]:
2 * pt(-abs(hln), df = T - 1)

In [ ]:
#checking ourselves
dm.test(e1 = L.mean_model, e2 = L.ar, power = 1) #We have already squared our errors, that's why power = 1

In [ ]:
# A small difference is only because we used unbiased estimator of variance, while in the function a biased one is used
biased_var <- function(x){
    m <- mean(x)
    result <- sum((x - m)^2)/length(x)
    result
}

d.mean/sqrt(biased_var(d)/T)*k; 2*pt(q = -abs(d.mean/sqrt(biased_var(d)/T)*k), df = T - 1)

## Multiple model comparison

- I hope that you remember lecture
- Anyway, a small reminder
    - We have `k+1` models
    - We are trying to test: $H_0: E[L(e^1_t)] = E[L(e^2_t)] = ... = E[L(e^{k+1}_t)]$
    - Basically, we have: $H_0: E[{\bf d_t}] = 0, \text{ where } {\bf d_t} = \left( d_{1,t}, d_{2, t}, ... , d_{k, t} \right), \text{ and } d_{j, t} = L(e^j_t) - L(e^{j+1}_t), j = 1...k$
    - Then, our statistic looks like: $T \cdot {\bf{\bar{d}}}' \cdot \hat{\Omega}^{-1} \cdot {\bf{\bar{d}}} \overset{a}{\rightarrow} \chi^2_k$
- Ok, let's do it!

- First, we need another model as we have only two: mean model and AR(1) model
- I decided to use MA(1) model for forecasting

- library `forecast` has its own `Arima()` function that differs from built-in `arima()` function:  
>If you want to choose the model yourself, use the `Arima()` function in R. There is another function `arima()` in R which also fits an ARIMA model. However, it does not allow for the constant c unless $d = 0$, and it does not return everything required for other functions in the `forecast` package to work. Finally, it does not allow the estimated model to be applied to new data (which is useful for checking forecast accuracy). Consequently, it is recommended that `Arima()` be used instead.

[source](https://otexts.com/fpp2/arima-r.html)

- I hope that you check the source above
- To put it simple, you need only two functions 

In [ ]:
# First, is fitting the model
test_model <- Arima(y = sp.ret, order = c(1, 0, 0))

In [ ]:
summary(test_model)

In [ ]:
str(test_model)

In [ ]:
# And the 1-step ahead forecasting from the model
test_pred <- forecast(object = test_model, h = 1)

In [ ]:
summary(test_pred)

In [ ]:
str(test_pred)

In [ ]:
# Probably, we want the point forecats value
test_pred$mean[1]

In [ ]:
# Now, let's get predictions for MA(1) model
y_pred_ma <- # initialiaze a vector

for (i in 1:N_OOS){
    y_train <- # keep your train data
    ma <- #fit the model on your train data
    y_pred_ma[i] <- #make a forecast
}

y_pred_ma <- as.xts(y_pred_ma, order.by = index(y_true))

In [ ]:
par(xpd = T)
plot(x = cbind(y_true, y_pred_mean, y_pred_ar, y_pred_ma), 
     main = 'Returns and predictions', 
     y = index(y_true), 
     col = c('black', 'red', 'blue', 'green'),
     lwd = c(1, 2, 2, 2))
legend('bottomleft', 
       legend = c('Returns', 'Mean model predictions', 'AR(1) model predictions', 'MA(1) model predictions'), 
       col = c('black', 'red', 'blue', 'green'),
       lty = c(1, 1, 1, 1),
       inset = 0.05)

In [ ]:
# let's compute MSE for MA(1) model also
mse_ma <- #your code

In [ ]:
mse_mean;mse_ar;mse_ma

In [ ]:
# computing squared forecasting errors
L.ma <- #your code

In [ ]:
# Let's compare the MA(1) model with the constant
# We've already computed DM test by ourselves, so we can legitimately use written function
dm.test(e1 = L.mean_model, e2 = L.ma, h = 1, power = 1)

In [ ]:
# And for curiosity do DM test for AR(1) and MA(1) models
dm.test(e1 = L.ar, e2 = L.ma, h = 1, power = 1)

**- What can you say?**

- Let's extend our testing to multiple models case
- I think it is enough to start with something simple as three models
- The main inspiration were taken from [here](https://cran.r-project.org/web/packages/multDM/index.html)
- Do not forget that as `R` is open source and you can always see what is going on inside the package
- The question is would you understand though, as in some cases authors are using, for example, `C++` inside some functions
- Anyway, this is not the case

In [ ]:
# the length of our forecats is basically the number of out-of-sample observations
T <- N_OOS

In [ ]:
# We have already calculated all losses
# We need to calculate differences
# We have three models -> we need two difference series
d_1 <- #difference between mean model and AR(1) model
d_2 <- #difference between AR(1) model and MA(1) model
d_multiple <- cbind(d_1, d_2)

In [ ]:
# Let's put them into one vector
d_multiple.mean <- #mean of the columns

- Ok, next is calculating a consistent estimator of its asymptotic variance
- As I wrote earlier, I will use [`multDM`](https://cran.r-project.org/web/packages/multDM/index.html) package code that implemented this procedure from 
- [R.S. Mariano and D. Preve (2012) Statistical tests for multiple forecast comparison, Journal of Econometrics 169, 123-130](https://www.researchgate.net/publication/230667169_Statistical_Tests_for_Multiple_Forecast_Comparison)

In [ ]:
omega <- #consistent variance estimator for 1-step ahead forecast

In [ ]:
# Our statistic is then
S <- #your code

In [ ]:
# use pchisq() function in order to get p-value
pchisq()

In [ ]:
# checking ourselves
library(multDM)
MDM.test(realized = y_true, 
         evaluated = t(cbind(y_pred_mean, y_pred_ar, y_pred_ma)), 
         q = 0, 
         statistic = 'S', 
         loss.type = 'SE')

## Giacomini and White test

- Anyway, it is quite a good idea to use not just AR(1) or MA(1) models but, for example, ARMA(1, 1) model
- Still, the question is how to compare predictions of, for example, AR(1) and ARMA(1, 1) models 
- I hope you understand why such question arises
- [Giacomini and White (2006)](https://12f404d7-f421-6d2a-2b6f-c6eeccdc238a.filesusr.com/ugd/55ca8b_acdd79b07ce300e2160466839f5d0d45.pdf) might help us

- Still, let's first get predictions from ARMA(1, 1) model

In [ ]:
y_pred_arma <- #initialiaze a vector

for (i in 1:N_OOS){
    y_train <- # keep your train data
    arma <- #fit the model
    y_pred_arma[i] <- #make a forecast
}

y_pred_arma <- as.xts(y_pred_arma, order.by = index(y_true))

In [ ]:
par(xpd = T)
plot(x = cbind(y_true, y_pred_mean, y_pred_ar, y_pred_ma, y_pred_arma), 
     main = 'Returns and predictions', 
     y = index(y_true), 
     col = 1:5,
     lwd = c(1, 2, 2, 2, 2))
legend('bottomleft', 
       legend = c('Returns', 'Mean model predictions', 'AR(1) model predictions', 'MA(1) model predictions', 'ARMA(1, 1) model predictions'), 
       col = 1:5,
       lty = c(1, 1, 1, 1),
       inset = 0.05)

In [ ]:
# let's compute MSE for MA(1) model also
mse_arma <- #your code

In [ ]:
mse_mean;mse_ar;mse_ma;mse_arma

In [ ]:
L.arma <- #calculate squared forecasting errors

d_gw <- #calculate the difference series between ARMA(1, 1) model and AR(1) model

- As were written in the lecture notes, it is a common practice to say that $h^{*}_t$ is a unit vector and past forecast error
- Please, be very careful with indices. It really does matter for this test 

In [ ]:
TT <- T-1
h <- #bing vector of ones of length (T-1) and difference series from 1 to (T-1)

In [ ]:
d_gw <- #re-write your difference series from 2 to T

In [ ]:
# As we have 1-step ahead forecast we can this

hL <- matrix(0, nrow = nrow(h), ncol = ncol(h))

for (i in 1:ncol(h)){
    hL[, i] <- # multiply your h and new difference series. Be careful with xts. Better to convert to vectors and multiply them
}

omega_gw <- #calculate consistent variance estimator

In [ ]:
gw <- #use formula for the statistic

In [ ]:
gw

In [ ]:
#calculate p-value
pchisq()

- For those of you who know `MatLab` you can check my translation from `MatLab` to `R` by using the original code of the authors from [here](http://www.runmycode.org/companion/view/88)